In [1]:
!pip install tensorflow opencv-python mediapipe sklearn matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install tensorflow-gpu

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  ERROR: Error [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software while executing command python setup.py egg_info
ERROR: Could not install packages due to an OSError: [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software



In [4]:
!pip install mediapipe

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install sklearn matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic    # Holistic Model
mp_drawing = mp.solutions.drawing_utils    # Drawing Utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245, 137, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245, 66, 250), thickness=2, circle_radius=2)
                             )

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        
        draw_styled_landmarks(image, results)

        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF== ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [7]:
len(results.left_hand_landmarks.landmark)

NameError: name 'results' is not defined

In [8]:
draw_landmarks(frame, results)

NameError: name 'frame' is not defined

In [5]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

NameError: name 'frame' is not defined

In [ ]:
results

In [7]:
all_landmarks = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    all_landmarks.append(test)

NameError: name 'results' is not defined

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose, face, lh, rh])

In [9]:
extract_keypoints(results).shape

NameError: name 'results' is not defined

In [7]:
DATA_PATH = os.path.join("MP_Data")
actions = np.array(['hello', 'thanks', 'iloveyou', 'what', 'bad'])
no_sequences = 30
sequence_length = 30

In [11]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [12]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)

                draw_styled_landmarks(image, results)
                
                if frame_num == 0:
                    cv2.putText(image, 'Starting Collection', (120, 200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting Frames for {} Video Number {}'.format(action, sequence), (15, 12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video NUmber {}'.format(action, sequence), (15, 12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                cv2.imshow('OpenCV Feed', image)
                
                if cv2.waitKey(10) & 0xFF== ord('q'):
                    break
cap.release()
cv2.destroyAllWindows()

In [13]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
!pip install scikit-learn

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2, 'what': 3, 'bad': 4}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
X = np.array(sequences)

In [13]:
y = to_categorical(labels).astype(int)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [15]:
x_train

array([[[ 0.54049248,  0.70565647, -0.82878858, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.54324812,  0.70637202, -0.73680532, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.54669946,  0.70642632, -0.7536242 , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.55602992,  0.70918143, -0.82941467, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.55549341,  0.71018344, -0.84439152, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.55496895,  0.71085781, -0.84464532, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.59732378,  0.73266059, -0.52541614, ...,  0.48335326,
          0.74499452, -0.04396825],
        [ 0.59731919,  0.73191971, -0.51664865, ...,  0.48491833,
          0.74757153, -0.04136304],
        [ 0.59731472,  0.73140758, -0.51139975, ...,  0.48621064,
          0.74859291, -0.04216007],
        ...,
        [ 0.60789526,  0.72381145, -0.4547587 , ...,  

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [9]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [10]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

NameError: name 'actions' is not defined

In [19]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [20]:
model.fit(x_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
5/5 [==============================] - 7s 55ms/step - loss: 5.2882 - categorical_accuracy: 0.2113
Epoch 2/1000
5/5 [==============================] - 0s 56ms/step - loss: 2.5436 - categorical_accuracy: 0.2042
Epoch 3/1000
5/5 [==============================] - 0s 58ms/step - loss: 1.7762 - categorical_accuracy: 0.2324
Epoch 4/1000
5/5 [==============================] - 0s 61ms/step - loss: 1.7332 - categorical_accuracy: 0.1761
Epoch 5/1000
5/5 [==============================] - 0s 59ms/step - loss: 1.6382 - categorical_accuracy: 0.1972
Epoch 6/1000
5/5 [==============================] - 0s 60ms/step - loss: 1.6297 - categorical_accuracy: 0.1831
Epoch 7/1000
5/5 [==============================] - 0s 60ms/step - loss: 1.6230 - categorical_accuracy: 0.1972
Epoch 8/1000
5/5 [==============================] - 0s 60ms/step - loss: 1.5795 - categorical_accuracy: 0.2817
Epoch 9/1000
5/5 [==============================] - 0s 57ms/step - loss: 1.5963 - categorical_accuracy: 0.2183
E

5/5 [==============================] - 0s 66ms/step - loss: 1.6075 - categorical_accuracy: 0.1972
Epoch 75/1000
5/5 [==============================] - 0s 65ms/step - loss: 1.6059 - categorical_accuracy: 0.2113
Epoch 76/1000
5/5 [==============================] - 0s 66ms/step - loss: 1.6036 - categorical_accuracy: 0.2394
Epoch 77/1000
5/5 [==============================] - 0s 64ms/step - loss: 1.6024 - categorical_accuracy: 0.2042
Epoch 78/1000
5/5 [==============================] - 0s 64ms/step - loss: 1.6008 - categorical_accuracy: 0.2113
Epoch 79/1000
5/5 [==============================] - 0s 63ms/step - loss: 1.5980 - categorical_accuracy: 0.2183
Epoch 80/1000
5/5 [==============================] - 0s 63ms/step - loss: 1.5923 - categorical_accuracy: 0.2042
Epoch 81/1000
5/5 [==============================] - 0s 64ms/step - loss: 1.5884 - categorical_accuracy: 0.2254
Epoch 82/1000
5/5 [==============================] - 0s 63ms/step - loss: 1.6062 - categorical_accuracy: 0.2113
Epoch 

5/5 [==============================] - 0s 67ms/step - loss: 1.5817 - categorical_accuracy: 0.3310
Epoch 147/1000
5/5 [==============================] - 0s 66ms/step - loss: 1.5654 - categorical_accuracy: 0.3521
Epoch 148/1000
5/5 [==============================] - 0s 67ms/step - loss: 1.5276 - categorical_accuracy: 0.3239
Epoch 149/1000
5/5 [==============================] - 0s 65ms/step - loss: 1.5584 - categorical_accuracy: 0.3310
Epoch 150/1000
5/5 [==============================] - 0s 66ms/step - loss: 1.5850 - categorical_accuracy: 0.2465
Epoch 151/1000
5/5 [==============================] - 0s 70ms/step - loss: 1.6016 - categorical_accuracy: 0.1972
Epoch 152/1000
5/5 [==============================] - 0s 69ms/step - loss: 1.6065 - categorical_accuracy: 0.1972
Epoch 153/1000
5/5 [==============================] - 0s 69ms/step - loss: 1.6069 - categorical_accuracy: 0.1972
Epoch 154/1000
5/5 [==============================] - 0s 69ms/step - loss: 1.6054 - categorical_accuracy: 0.197

5/5 [==============================] - 0s 65ms/step - loss: 1.5967 - categorical_accuracy: 0.2746
Epoch 219/1000
5/5 [==============================] - 0s 68ms/step - loss: 1.5584 - categorical_accuracy: 0.2817
Epoch 220/1000
5/5 [==============================] - 0s 65ms/step - loss: 1.5620 - categorical_accuracy: 0.3099
Epoch 221/1000
5/5 [==============================] - 0s 69ms/step - loss: 1.6042 - categorical_accuracy: 0.2254
Epoch 222/1000
5/5 [==============================] - 0s 65ms/step - loss: 1.6056 - categorical_accuracy: 0.2394
Epoch 223/1000
5/5 [==============================] - 0s 69ms/step - loss: 1.6028 - categorical_accuracy: 0.2535
Epoch 224/1000
5/5 [==============================] - 0s 66ms/step - loss: 1.5968 - categorical_accuracy: 0.2254
Epoch 225/1000
5/5 [==============================] - 0s 68ms/step - loss: 1.5864 - categorical_accuracy: 0.2465
Epoch 226/1000
5/5 [==============================] - 0s 67ms/step - loss: 1.5612 - categorical_accuracy: 0.267

5/5 [==============================] - 0s 68ms/step - loss: 1.5849 - categorical_accuracy: 0.1972
Epoch 291/1000
5/5 [==============================] - 0s 68ms/step - loss: 1.5616 - categorical_accuracy: 0.2254
Epoch 292/1000
5/5 [==============================] - 0s 68ms/step - loss: 1.6201 - categorical_accuracy: 0.1901
Epoch 293/1000
5/5 [==============================] - 0s 67ms/step - loss: 1.6178 - categorical_accuracy: 0.1620
Epoch 294/1000
5/5 [==============================] - 0s 67ms/step - loss: 1.6065 - categorical_accuracy: 0.2042
Epoch 295/1000
5/5 [==============================] - 0s 68ms/step - loss: 1.6086 - categorical_accuracy: 0.2042
Epoch 296/1000
5/5 [==============================] - 0s 68ms/step - loss: 1.6049 - categorical_accuracy: 0.2042
Epoch 297/1000
5/5 [==============================] - 0s 70ms/step - loss: 1.6018 - categorical_accuracy: 0.2042
Epoch 298/1000
5/5 [==============================] - 0s 70ms/step - loss: 1.6051 - categorical_accuracy: 0.204

...

5/5 [==============================] - 0s 70ms/step - loss: 0.7521 - categorical_accuracy: 0.6901
Epoch 651/1000
5/5 [==============================] - 0s 72ms/step - loss: 0.7753 - categorical_accuracy: 0.6761
Epoch 652/1000
5/5 [==============================] - 0s 72ms/step - loss: 0.8115 - categorical_accuracy: 0.6901
Epoch 653/1000
5/5 [==============================] - 0s 71ms/step - loss: 0.6525 - categorical_accuracy: 0.7535
Epoch 654/1000
5/5 [==============================] - 0s 71ms/step - loss: 0.7851 - categorical_accuracy: 0.6338
Epoch 655/1000
5/5 [==============================] - 0s 72ms/step - loss: 0.7370 - categorical_accuracy: 0.7113
Epoch 656/1000
5/5 [==============================] - 0s 71ms/step - loss: 0.5768 - categorical_accuracy: 0.7817
Epoch 657/1000
5/5 [==============================] - 0s 70ms/step - loss: 0.6084 - categorical_accuracy: 0.7676
Epoch 658/1000
5/5 [==============================] - 0s 72ms/step - loss: 0.5824 - categorical_accuracy: 0.767

5/5 [==============================] - 0s 73ms/step - loss: 0.4796 - categorical_accuracy: 0.7958
Epoch 723/1000
5/5 [==============================] - 0s 74ms/step - loss: 0.5075 - categorical_accuracy: 0.8028
Epoch 724/1000
5/5 [==============================] - 0s 73ms/step - loss: 0.5532 - categorical_accuracy: 0.7606
Epoch 725/1000
5/5 [==============================] - 0s 73ms/step - loss: 0.6201 - categorical_accuracy: 0.7113
Epoch 726/1000
5/5 [==============================] - 0s 73ms/step - loss: 0.5126 - categorical_accuracy: 0.7746
Epoch 727/1000
5/5 [==============================] - 0s 73ms/step - loss: 0.7629 - categorical_accuracy: 0.7183
Epoch 728/1000
5/5 [==============================] - 0s 74ms/step - loss: 0.9557 - categorical_accuracy: 0.5423
Epoch 729/1000
5/5 [==============================] - 0s 70ms/step - loss: 1.4849 - categorical_accuracy: 0.4507
Epoch 730/1000
5/5 [==============================] - 0s 71ms/step - loss: 1.2997 - categorical_accuracy: 0.542

5/5 [==============================] - 0s 73ms/step - loss: 0.7629 - categorical_accuracy: 0.6690
Epoch 795/1000
5/5 [==============================] - 0s 73ms/step - loss: 0.6227 - categorical_accuracy: 0.7113
Epoch 796/1000
5/5 [==============================] - 0s 72ms/step - loss: 0.5442 - categorical_accuracy: 0.7535
Epoch 797/1000
5/5 [==============================] - 0s 73ms/step - loss: 0.5275 - categorical_accuracy: 0.7746
Epoch 798/1000
5/5 [==============================] - 0s 74ms/step - loss: 0.4785 - categorical_accuracy: 0.7887
Epoch 799/1000
5/5 [==============================] - 0s 74ms/step - loss: 0.4561 - categorical_accuracy: 0.8028
Epoch 800/1000
5/5 [==============================] - 0s 73ms/step - loss: 0.4460 - categorical_accuracy: 0.8169
Epoch 801/1000
5/5 [==============================] - 0s 75ms/step - loss: 0.4459 - categorical_accuracy: 0.8099
Epoch 802/1000
5/5 [==============================] - 0s 75ms/step - loss: 0.4211 - categorical_accuracy: 0.816

5/5 [==============================] - 0s 73ms/step - loss: 0.5637 - categorical_accuracy: 0.7676
Epoch 867/1000
5/5 [==============================] - 0s 72ms/step - loss: 0.4750 - categorical_accuracy: 0.8028
Epoch 868/1000
5/5 [==============================] - 0s 72ms/step - loss: 0.4661 - categorical_accuracy: 0.7958
Epoch 869/1000
5/5 [==============================] - 0s 72ms/step - loss: 0.4194 - categorical_accuracy: 0.8310
Epoch 870/1000
5/5 [==============================] - 0s 73ms/step - loss: 0.4212 - categorical_accuracy: 0.8099
Epoch 871/1000
5/5 [==============================] - 0s 75ms/step - loss: 0.4218 - categorical_accuracy: 0.8310
Epoch 872/1000
5/5 [==============================] - 0s 74ms/step - loss: 0.4061 - categorical_accuracy: 0.8239
Epoch 873/1000
5/5 [==============================] - 0s 73ms/step - loss: 0.4109 - categorical_accuracy: 0.8451
Epoch 874/1000
5/5 [==============================] - 0s 73ms/step - loss: 0.4067 - categorical_accuracy: 0.823

5/5 [==============================] - 0s 73ms/step - loss: 0.5156 - categorical_accuracy: 0.7606
Epoch 939/1000
5/5 [==============================] - 0s 75ms/step - loss: 0.4333 - categorical_accuracy: 0.8239
Epoch 940/1000
5/5 [==============================] - 0s 74ms/step - loss: 0.3786 - categorical_accuracy: 0.8521
Epoch 941/1000
5/5 [==============================] - 0s 74ms/step - loss: 0.3997 - categorical_accuracy: 0.8239
Epoch 942/1000
5/5 [==============================] - 0s 74ms/step - loss: 0.5052 - categorical_accuracy: 0.7606
Epoch 943/1000
5/5 [==============================] - 0s 74ms/step - loss: 0.5905 - categorical_accuracy: 0.7254
Epoch 944/1000
5/5 [==============================] - 0s 73ms/step - loss: 0.4222 - categorical_accuracy: 0.8028
Epoch 945/1000
5/5 [==============================] - 0s 72ms/step - loss: 0.4359 - categorical_accuracy: 0.8239
Epoch 946/1000
5/5 [==============================] - 0s 73ms/step - loss: 0.4141 - categorical_accuracy: 0.823

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 5)                 165       
                                                                 
Total params: 596741 (2.28 MB)
Trainable params: 596741 

In [7]:
res = model.predict(x_test)

NameError: name 'model' is not defined

In [23]:
actions[np.argmax(res[6])]

'bad'

In [24]:
model.save('action.keras')

In [25]:
model.load_weights('action.keras')

In [26]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [27]:
yhat = model.predict(x_train)

5/5 [==============================] - 0s 23ms/step


In [28]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [29]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[113,   1],
        [  3,  25]],

       [[110,   3],
        [  7,  22]],

       [[105,   8],
        [  3,  26]],

       [[109,   5],
        [  0,  28]],

       [[114,   0],
        [  4,  24]]], dtype=int64)

In [30]:
accuracy_score(ytrue, yhat)

0.8802816901408451

In [31]:
accuracy_score(ytrue, yhat)

0.8802816901408451

In [1]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [6]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

NameError: name 'res' is not defined

<Figure size 1800x1800 with 0 Axes>

In [ ]:
sequence.reverse()

In [ ]:
len(sequence)

In [ ]:
sequence.append('def')

In [ ]:
sequence.reverse()

In [ ]:
sequence[-30:]

In [32]:
sequence = []
sentence = []
threshold = 0.4

# Load your trained model here
# model = load_your_model_here()

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)

        keypoints = extract_keypoints(results)
        sequence.insert(0, keypoints)
        sequence = sequence[:30]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

            if np.any(res > threshold):
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

        if len(sentence) > 5:
            sentence = sentence[-5:]

        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 18ms/step
what
1/1 [==============================] - 0s 18ms/step
what
1/1 [==============================] - 0s 16ms/step
what
1/1 [==============================] - 0s 17ms/step
what
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 17ms/step
thanks
1/1 [==============================] - 0s 17ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 17ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 17ms/step
hello
1/1 [==

1/1 [==============================] - 0s 18ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 18ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 17ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 18ms/step
hello
1/1 [==============================] - 0s 17ms/step
hello
1/1 [==============================] - 0s 17ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 18ms/step
bad
1/1 [==============================] - 0s 16ms/step
bad
1/1 [=============

1/1 [==============================] - 0s 17ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 17ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 17ms/step
thanks
1/1 [==============================] - 0s 22ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 17ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 18ms/step
thanks
1/1 [==============================] - 0s 17ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 17ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 16ms/step
than

1/1 [==============================] - 0s 15ms/step
iloveyou
1/1 [==============================] - 0s 18ms/step
thanks
1/1 [==============================] - 0s 19ms/step
thanks
1/1 [==============================] - 0s 17ms/step
thanks
1/1 [==============================] - 0s 17ms/step
thanks
1/1 [==============================] - 0s 17ms/step
thanks
1/1 [==============================] - 0s 17ms/step
thanks
1/1 [==============================] - 0s 18ms/step
thanks
1/1 [==============================] - 0s 17ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 18ms/step
thanks
1/1 [==============================] - 0s 17ms/step
thanks
1/1 [==============================] - 0s 16ms/step
thanks
1/1 [==============================] - 0s 16ms/step
iloveyou
1/1 [==============================] - 0s 17ms/step
iloveyou
1/1 [==============================] - 0s 17ms/ste

1/1 [==============================] - 0s 17ms/step
hello
1/1 [==============================] - 0s 17ms/step
hello
1/1 [==============================] - 0s 17ms/step
hello
1/1 [==============================] - 0s 15ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 17ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [==============================] - 0s 15ms/step
hello
1/1 [==============================] - 0s 17ms/step
hello
1/1 [==============================] - 0s 17ms/step
hello
1/1 [==============================] - 0s 15ms/step
hello
1/1 [==============================] - 0s 17ms/step
hello
1/1 [==============================] - 0s 17ms/step
hello
1/1 [==============================] - 0s 15ms/step
hello
1/1 [==============================] - 0s 15ms/step
hello
1/1 [==============================] - 0s 16ms/step
hello
1/1 [=========

...

1/1 [==============================] - 0s 23ms/step
iloveyou
1/1 [==============================] - 0s 23ms/step
iloveyou
1/1 [==============================] - 0s 24ms/step
iloveyou
1/1 [==============================] - 0s 22ms/step
iloveyou
1/1 [==============================] - 0s 23ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 24ms/step
iloveyou
1/1 [==============================] - 0s 23ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 27ms/step
iloveyou
1/1 [==============================] - 0s 20ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 18ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 21ms/step
iloveyou
1/1 [===================

1/1 [==============================] - 0s 24ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 18ms/step
iloveyou
1/1 [==============================] - 0s 18ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 24ms/step
iloveyou
1/1 [==============================] - 0s 18ms/step
iloveyou
1/1 [==============================] - 0s 18ms/step
iloveyou
1/1 [==============================] - 0s 26ms/step
iloveyou
1/1 [==============================] - 0s 21ms/step
iloveyou
1/1 [==============================] - 0s 20ms/step
iloveyou
1/1 [==============================] - 0s 24ms/step
iloveyou
1/1 [==============================] - 0s 23ms/step
iloveyou
1/1 [==============================] - 0s 22ms/step
iloveyou
1/1 [==============================] - 0s 20ms/step
iloveyou
1/1 [==============================] - 0s 23ms/step
iloveyou
1/1 [===================

1/1 [==============================] - 0s 20ms/step
hello
1/1 [==============================] - 0s 29ms/step
hello
1/1 [==============================] - 0s 26ms/step
hello
1/1 [==============================] - 0s 20ms/step
hello
1/1 [==============================] - 0s 18ms/step
hello
1/1 [==============================] - 0s 25ms/step
hello
1/1 [==============================] - 0s 24ms/step
hello
1/1 [==============================] - 0s 20ms/step
hello
1/1 [==============================] - 0s 23ms/step
hello
1/1 [==============================] - 0s 25ms/step
hello
1/1 [==============================] - 0s 18ms/step
hello
1/1 [==============================] - 0s 18ms/step
hello
1/1 [==============================] - 0s 23ms/step
hello
1/1 [==============================] - 0s 19ms/step
hello
1/1 [==============================] - 0s 19ms/step
hello
1/1 [==============================] - 0s 30ms/step
hello
1/1 [==============================] - 0s 20ms/step
hello
1/1 [=========

1/1 [==============================] - 0s 19ms/step
hello
1/1 [==============================] - 0s 18ms/step
hello
1/1 [==============================] - 0s 19ms/step
hello
1/1 [==============================] - 0s 21ms/step
hello
1/1 [==============================] - 0s 17ms/step
hello
1/1 [==============================] - 0s 22ms/step
hello
1/1 [==============================] - 0s 23ms/step
hello
1/1 [==============================] - 0s 23ms/step
hello
1/1 [==============================] - 0s 18ms/step
hello
1/1 [==============================] - 0s 18ms/step
hello
1/1 [==============================] - 0s 18ms/step
hello
1/1 [==============================] - 0s 18ms/step
bad
1/1 [==============================] - 0s 17ms/step
bad
1/1 [==============================] - 0s 17ms/step
bad
1/1 [==============================] - 0s 24ms/step
bad
1/1 [==============================] - 0s 23ms/step
bad
1/1 [==============================] - 0s 23ms/step
bad
1/1 [=====================

KeyboardInterrupt: 

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
res[np.argmax(res)] > threshold

In [ ]:
(num_sequences,30,1662)

In [ ]:
model.predict(np.expand_dims(X_test[0], axis=0))